# Week 3 Project. Segmenting and Clustering Toronto.
## Choosing the most suitable Neighbourhood.

### 1st. Importation of main libraries to the Jupiter Notebook.
We will make the imports in various cells as the speed for connection is not always available.

In [1]:
import pandas as pd
import numpy as np
import urllib.request

### 2nd. Importation of special libraries for visualization and coordinates

In [2]:
!conda install -c conda-forge geopy --yes #Needed to reinstall in order to work
from geopy.geocoders import Nominatim
import folium
import requests

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [3]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### 3rd. Installation of BeatifulSoup.
#### this is a library that will help us to download easily the table within the Wikipedia article

In [ ]:
conda install -c anaconda beautifulsoup4 #Needed to be installeed to make BeautifulSoup work

Solving environment: - 

In [ ]:
from bs4 import BeautifulSoup

### Now we will download the Wikipedia page with urllib library utilities.

In [ ]:
webpageurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = urllib.request.urlopen(webpageurl)
CanadaZIPsWeb = req.read().decode()


### We are now using BeautifulSoup to extract the table.

In [ ]:
CanadaZIPsoup = BeautifulSoup(CanadaZIPsWeb, 'html.parser')
CanadaZIPtable = CanadaZIPsoup.find('table', class_='sortable')


### Now we create a Panda Dataframe with the structure needed in order to append the data extracted with BeautifulSoup

In [ ]:
dfCanadaNeigh=pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
dfCanadaNeigh

### All data inside the table is after 'td' so we first create the list with all 'td'

In [ ]:
ValuesString=CanadaZIPtable.find_all('td')
ValuesString

#### Now we use a for in order to read the string and put it into the dataframe. We will add the data without the special characters.

In [ ]:
for i in range(1,int(len(ValuesString)/3)): 
    dfCanadaNeigh=dfCanadaNeigh.append({'Postcode':ValuesString[3*i+0].text.strip('[]\n'),'Borough':ValuesString[3*i+1].text.strip('[]\n'),'Neighbourhood':ValuesString[3*i+2].text.strip('[]\n')},ignore_index=True)
dfCanadaNeigh.tail()

##### We just perform a safety copy and generate a new one sorted and grouped.

In [ ]:
dfCanadaNeighOriginal=dfCanadaNeigh

#### Now we join postcodes, neighbourhood as requested

In [ ]:
index_cols = dfCanadaNeigh.columns.tolist()
index_cols.remove("Neighbourhood") 
dfCanadaNeigh = dfCanadaNeigh.groupby(index_cols)["Neighbourhood"].apply(list)
dfCanadaNeigh = dfCanadaNeigh.reset_index()


In [ ]:
dfCanadaNeigh.tail()

In [ ]:
dfCanadaNeigh['Neighbourhood']=dfCanadaNeigh['Neighbourhood'].astype(str).str.replace(r"[\[\]']", '')
dfCanadaNeigh.head()

In [ ]:
dfCanadaNeigh

#### For last, we will replace the "Not assigned", for the 'Borough' column in the same row.

In [ ]:
dfCanadaNeighCopy=dfCanadaNeigh
dfCanadaNeigh['Neighbourhood'].replace("Not assigned", dfCanadaNeigh['Borough'], inplace=True)
dfCanadaNeigh.head()

In [ ]:
FinalCanadadf = dfCanadaNeigh[dfCanadaNeigh.Borough != 'Not assigned'] #this operation creates a new df without the Borough not yet assigned.
FinalCanadadf

## Now we import the Geospatial data from given in the assignment

In [ ]:
CanadaGeodf = pd.read_csv('https://cocl.us/Geospatial_data')
CanadaGeodf.head()

#### Now we merge the data downloaded from the CSV file into the previous dataframe of the neighbourhoods

In [ ]:
dfCanadaNeigh = pd.merge(FinalCanadadf, CanadaGeodf, how='left',
        left_on='Postcode', right_on='Postal Code', validate="1:1")
dfCanadaNeigh.head()


#### The postal code has been merged as well, so we take that column out

In [ ]:
FinalCanadadf.drop(labels='Postal Code', axis=1, inplace=True)
FinalCanadadf.head()

### Now we export the df as a CSV file in order to make the 3rd part of the code leaner

In [ ]:
FinalCanadadf.to_csv('CanadaZIPCodeswithPosition.csv')